In [12]:
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu
import matplotlib.pyplot as plt
%matplotlib inline
colors = ['lightblue', 'pink']
feature_keys = ['total words','total uniq', 'total wpt','D uniq','D words',\
                'D wpt','D pos_avg','P uniq','P words','P wpt','P pos_avg']
df_phase1  = pd.read_csv('phase1_lect_ata_new_df.csv')
df_phase2  = pd.read_csv('phase2_lect_ata_new_df.csv')
df = pd.concat([df_phase1,df_phase2])

df_sent  = pd.read_csv('all_w_sent.csv',encoding = "ISO-8859-1")
df_liwc  = pd.read_csv('all_w_liwc.csv',encoding = "ISO-8859-1")
#df_liwc = df_liwc.groupby(['Filename']).mean()
#df_sent = df_sent.groupby(['Filename']).mean()
def get_male_female_files(p_male,d_male):
	phase1_files = df_phase1[(df_phase1['isPatientMale']==p_male)&(df_phase1['isDoctorMale']==d_male)]['Filename'].tolist()
	phase2_files = df_phase2[(df_phase2['isPatientMale']==p_male)&(df_phase2['isDoctorMale']==d_male)]['Filename'].tolist()
	return phase1_files+phase2_files

male_doctor_file_names = get_male_female_files(0,1)+ get_male_female_files(1,1)
female_doctor_file_names = get_male_female_files(0,0)+ get_male_female_files(1,0)
male_patient_file_names = get_male_female_files(1,0)+ get_male_female_files(1,1)
female_patient_file_names = get_male_female_files(0,0)+ get_male_female_files(0,1)

maleD_femaleP_file_names = get_male_female_files(0,1)
maleD_maleP_file_names = get_male_female_files(1,1)
femaleD_femaleP_file_names = get_male_female_files(0,0)
femaleD_maleP_file_names = get_male_female_files(1,0)
df['C pos_avg'].dropna().count()

280

<h2>Difference in Male-female doctors' liwc features</h2>

In [2]:
liwc_categories = df_liwc.keys()[3:]
liwc_categories = ['Achievement','Affective','Anger','Anxiety','Biological','Body','Causation','Certainty','Cognitive',\
                 'Discrepancy','Family','Feeling','Fillers','Friends','Future tense','Health','Hearing','Home',\
                 'Humans','Insight','Leisure','Money','Negative Emotion','Positive emotion','Religion','Sadness',\
                 'Sexual','Social','Swear','Time','death@Death']
result = dict()
result['Feature'] = []
result['Male Mean'] = []
result['Female Mean'] = []
result['MWU P'] = []
result['Cliff\'s d'] = []
for cat in liwc_categories:
    male_feat = df_liwc[df_liwc['Filename'].isin(male_doctor_file_names)].groupby(['Filename']).sum()
    male_feat = male_feat[cat]/male_feat['n_words']
    female_feat = df_liwc[df_liwc['Filename'].isin(female_doctor_file_names)].groupby(['Filename']).sum()
    female_feat = female_feat[cat]/female_feat['n_words']
    
    U, p = mannwhitneyu(male_feat,female_feat)
    d = 2*U/(len(male_feat)*len(female_feat)) -1
    result['Feature'].append(cat)
    result['Male Mean'].append(male_feat.mean())
    result['Female Mean'].append(female_feat.mean())
    result['MWU P'].append(p*43)
    result['Cliff\'s d'].append(abs(d))
    #print(cat,male_feat.mean(),female_feat.mean(),p,d)
result = pd.DataFrame(result)
result = result.sort_values(['MWU P'])

In [3]:
result[result['MWU P'] < 0.05][['Feature','Male Mean', 'Female Mean', 'MWU P', 'Cliff\'s d']]

,Feature,Male Mean,Female Mean,MWU P,Cliff's d
8,Cognitive,0.152319,0.160829,0.000080,0.290443
12,Fillers,0.004838,0.005805,0.000305,0.272564
19,Insight,0.016821,0.018784,0.000482,0.266203
9,Discrepancy,0.023531,0.025765,0.002837,0.240049


<h2>Among Male Doctor's which liwc features are different among male-female patients </h2>

In [4]:
liwc_categories = ['Achievement','Affective','Anger','Anxiety','Biological','Body','Causation','Certainty','Cognitive',\
                 'Discrepancy','Family','Feeling','Fillers','Friends','Future tense','Health','Hearing','Home',\
                 'Humans','Insight','Leisure','Money','Negative Emotion','Positive emotion','Religion','Sadness',\
                 'Sexual','Social','Swear','Time','death@Death']
result = dict()
result['Feature'] = []
result['Male Mean'] = []
result['Female Mean'] = []
result['MWU P'] = []
result['Cliff\'s d'] = []
for cat in liwc_categories:
    male_feat = df_liwc[df_liwc['Filename'].isin(maleD_maleP_file_names)].groupby(['Filename']).sum()
    male_feat = male_feat[cat]/male_feat['n_words']
    female_feat = df_liwc[df_liwc['Filename'].isin(maleD_femaleP_file_names)].groupby(['Filename']).sum()
    female_feat = female_feat[cat]/female_feat['n_words']
    U, p = mannwhitneyu(male_feat,female_feat)
    d = 2*U/(len(male_feat)*len(female_feat)) -1
    result['Feature'].append(cat)
    result['Male Mean'].append(male_feat.mean())
    result['Female Mean'].append(female_feat.mean())
    result['MWU P'].append(p*43)
    result['Cliff\'s d'].append(abs(d))
    #print(cat,male_feat.mean(),female_feat.mean(),p,d)
result = pd.DataFrame(result)
result = result.sort_values(['MWU P'])
result[result['MWU P'] < 0.05][['Feature','Male Mean', 'Female Mean', 'MWU P', 'Cliff\'s d']]

,Feature,Male Mean,Female Mean,MWU P,Cliff's d


<h2>Among Female Doctor's which liwc features are different among male-female patients </h2>

In [5]:
liwc_categories = ['Achievement','Affective','Anger','Anxiety','Biological','Body','Causation','Certainty','Cognitive',\
                 'Discrepancy','Family','Feeling','Fillers','Friends','Future tense','Health','Hearing','Home',\
                 'Humans','Insight','Leisure','Money','Negative Emotion','Positive emotion','Religion','Sadness',\
                 'Sexual','Social','Swear','Time','death@Death']
result = dict()
result['Feature'] = []
result['Male Mean'] = []
result['Female Mean'] = []
result['MWU P'] = []
result['Cliff\'s d'] = []
for cat in liwc_categories:
    male_feat = df_liwc[df_liwc['Filename'].isin(femaleD_maleP_file_names)].groupby(['Filename']).sum()
    female_feat = df_liwc[df_liwc['Filename'].isin(femaleD_femaleP_file_names)].groupby(['Filename']).sum()
    male_feat = male_feat[cat]/male_feat['n_words']
    female_feat = female_feat[cat]/female_feat['n_words']
    U, p = mannwhitneyu(male_feat,female_feat)
    d = 2*U/(len(male_feat)*len(female_feat)) -1
    result['Feature'].append(cat)
    result['Male Mean'].append(male_feat.mean())
    result['Female Mean'].append(female_feat.mean())
    result['MWU P'].append(p*43)
    result['Cliff\'s d'].append(abs(d))
    #print(cat,male_feat.mean(),female_feat.mean(),p,d)
result = pd.DataFrame(result)
result = result.sort_values(['MWU P'])
result[result['MWU P'] < 0.05][['Feature','Male Mean', 'Female Mean', 'MWU P', 'Cliff\'s d']]

,Feature,Male Mean,Female Mean,MWU P,Cliff's d
26,Sexual,0.000348,0.000425,0.002892,0.411131


<h2>Difference in Male-female doctors' features</h2>


In [9]:
from scipy.stats import ttest_ind
feat_result = dict()
feat_result['Feature'] = []
feat_result['Male Mean'] = []
feat_result['Female Mean'] = []
feat_result['MWU P'] = []
feat_result['Cliff\'s d'] = []
for cat in feature_keys:
    male_feat = df[df['Filename'].isin(male_doctor_file_names)].groupby(['Filename']).mean()[cat]
    female_feat = df[df['Filename'].isin(female_doctor_file_names)].groupby(['Filename']).mean()[cat]
    U, p = mannwhitneyu(male_feat,female_feat)
    #p = ttest_ind(male_feat,female_feat)[1]
    d = 2*U/(len(male_feat)*len(female_feat)) -1
    feat_result['Feature'].append(cat)
    feat_result['Male Mean'].append(male_feat.mean())
    feat_result['Female Mean'].append(female_feat.mean())
    feat_result['MWU P'].append(p*43)
    feat_result['Cliff\'s d'].append(abs(d))
    #print(cat,male_feat.mean(),female_feat.mean(),p,d)
feat_result = pd.DataFrame(feat_result)
feat_result = feat_result.sort_values(['MWU P'])
feat_result[feat_result['MWU P'] < 1][['Feature','Male Mean', 'Female Mean', 'MWU P', 'Cliff\'s d']]

,Feature,Male Mean,Female Mean,MWU P,Cliff's d
4,D words,1664.611765,2410.606299,4.929514e-10,0.419824
3,D uniq,417.996078,524.291339,1.575202e-08,0.386722
0,total words,3078.094118,4360.039370,3.257258e-08,0.379435
1,total uniq,607.450980,739.055118,7.474421e-07,0.346333
8,P words,965.588235,1337.582677,7.308937e-04,0.260275
5,D wpt,16.316406,17.931134,1.717598e-03,0.247707
2,total wpt,12.333780,13.180371,6.294386e-03,0.227420
7,P uniq,293.701961,348.669291,6.945004e-03,0.225814
6,D pos_avg,0.207626,0.223381,5.131158e-02,0.190736


<h2>Among Male Doctor's which features are different among male-female patients </h2>

In [7]:
feat_result = dict()
feat_result['Feature'] = []
feat_result['Male Mean'] = []
feat_result['Female Mean'] = []
feat_result['MWU P'] = []
feat_result['Cliff\'s d'] = []
for cat in feature_keys:
    male_feat = df[df['Filename'].isin(maleD_maleP_file_names)].groupby(['Filename']).mean()[cat]
    female_feat = df[df['Filename'].isin(maleD_femaleP_file_names)].groupby(['Filename']).mean()[cat]
    U, p = mannwhitneyu(male_feat,female_feat)
    d = 2*U/(len(male_feat)*len(female_feat)) -1
    feat_result['Feature'].append(cat)
    feat_result['Male Mean'].append(male_feat.mean())
    feat_result['Female Mean'].append(female_feat.mean())
    feat_result['MWU P'].append(p*43)
    feat_result['Cliff\'s d'].append(abs(d))
    #print(cat,male_feat.mean(),female_feat.mean(),p,d)
feat_result = pd.DataFrame(feat_result)
feat_result = feat_result.sort_values(['MWU P'])
feat_result[feat_result['MWU P'] < 0.05][['Feature','Male Mean', 'Female Mean', 'MWU P', 'Cliff\'s d']]

,Feature,Male Mean,Female Mean,MWU P,Cliff's d


<h2>Among Female Doctor's which features are different among male-female patients </h2>

In [8]:
feat_result = dict()
feat_result['Feature'] = []
feat_result['Male Mean'] = []
feat_result['Female Mean'] = []
feat_result['MWU P'] = []
feat_result['Cliff\'s d'] = []
for cat in feature_keys:
    male_feat = df[df['Filename'].isin(femaleD_maleP_file_names)].groupby(['Filename']).mean()[cat]
    female_feat = df[df['Filename'].isin(femaleD_femaleP_file_names)].groupby(['Filename']).mean()[cat]
    U, p = mannwhitneyu(male_feat,female_feat)
    d = 2*U/(len(male_feat)*len(female_feat)) -1
    feat_result['Feature'].append(cat)
    feat_result['Male Mean'].append(male_feat.mean())
    feat_result['Female Mean'].append(female_feat.mean())
    feat_result['MWU P'].append(p*43)
    feat_result['Cliff\'s d'].append(abs(d))
    #print(cat,male_feat.mean(),female_feat.mean(),p,d)
feat_result = pd.DataFrame(feat_result)
feat_result = feat_result.sort_values(['MWU P'])
feat_result[feat_result['MWU P'] < 0.05][['Feature','Male Mean', 'Female Mean', 'MWU P', 'Cliff\'s d']]

,Feature,Male Mean,Female Mean,MWU P,Cliff's d
